In [1]:
import xlrd
import pandas as pd
import os, sys

In [2]:
def process_file(filepath):
    print(filepath)
    wb = xlrd.open_workbook(filepath)  # Open workbook
    
    subj_data = wb.sheet_by_index(1)  # Open Sheet2 which contains subject data
    height = subj_data.cell_value(0, 1)
    floor_to_shank_imu = subj_data.cell_value(1, 1)
    floor_to_knee = subj_data.cell_value(2, 1)
    floor_to_hip = subj_data.cell_value(3, 1)
    knee_to_hip_imu = subj_data.cell_value(4, 1)
    weight = subj_data.cell_value(5, 1)
    
    imu_data = wb.sheet_by_index(0)  # Open Sheet1 which contains IMU Data
    num_data_points = imu_data.nrows
    compiled_data = {'Time': imu_data.col_values(1),
                     'Time Between Readings': imu_data.col_values(2),
                     'Shank X Gravity Vector': [x * -1 for x in imu_data.col_values(4)], 
                     'Shank Y Gravity Vector': [x * -1 for x in imu_data.col_values(5)], 
                     'Shank Z Gravity Vector': imu_data.col_values(6),
                     'Shank X Gyroscope Vector': [x * -1 for x in imu_data.col_values(8)],
                     'Shank Y Gyroscope Vector': [x * -1 for x in imu_data.col_values(9)],
                     'Shank Z Gyroscope Vector': imu_data.col_values(10),
                     'Shank X Magnetometer Vector': [x * -1 for x in imu_data.col_values(12)],
                     'Shank Y Magnetometer Vector': [x * -1 for x in imu_data.col_values(13)],
                     'Shank Z Magnetometer Vector': imu_data.col_values(14),
                     'Shank X Euler Vector': [x * -1 for x in imu_data.col_values(16)],
                     'Shank Y Euler Vector': [x * -1 for x in imu_data.col_values(17)],
                     'Shank Z Euler Vector': imu_data.col_values(18),
                     'Shank X Acceleration Vector': [x * -1 for x in imu_data.col_values(20)],
                     'Shank Y Acceleration Vector': [x * -1 for x in imu_data.col_values(21)],
                     'Shank Z Acceleration Vector': imu_data.col_values(22),
                     'Shank X Linear Acceleration Vector': [x * -1 for x in imu_data.col_values(24)],
                     'Shank Y Linear Acceleration Vector': [x * -1 for x in imu_data.col_values(25)],
                     'Shank Z Linear Acceleration Vector': imu_data.col_values(26),
                     'Shank W Quaternion': imu_data.col_values(28),
                     'Shank X Quaternion': imu_data.col_values(29),
                     'Shank Y Quaternion': imu_data.col_values(30),
                     'Shank Z Quaternion': imu_data.col_values(31),
                     'Thigh X Gravity Vector': imu_data.col_values(34), 
                     'Thigh Y Gravity Vector': imu_data.col_values(33), 
                     'Thigh Z Gravity Vector': [x * -1 for x in imu_data.col_values(35)],
                     'Thigh X Gyroscope Vector': imu_data.col_values(38),
                     'Thigh Y Gyroscope Vector': imu_data.col_values(37),
                     'Thigh Z Gyroscope Vector': [x * -1 for x in imu_data.col_values(39)],
                     'Thigh X Magnetometer Vector': imu_data.col_values(42),
                     'Thigh Y Magnetometer Vector': imu_data.col_values(41),
                     'Thigh Z Magnetometer Vector': [x * -1 for x in imu_data.col_values(43)],
                     'Thigh X Euler Vector': imu_data.col_values(46),
                     'Thigh Y Euler Vector': imu_data.col_values(45),
                     'Thigh Z Euler Vector': [x * -1 for x in imu_data.col_values(47)],
                     'Thigh X Acceleration Vector': imu_data.col_values(50),
                     'Thigh Y Acceleration Vector': imu_data.col_values(49),
                     'Thigh Z Acceleration Vector': [x * -1 for x in imu_data.col_values(51)],
                     'Thigh X Linear Acceleration Vector': imu_data.col_values(54),
                     'Thigh Y Linear Acceleration Vector': imu_data.col_values(53),
                     'Thigh Z Linear Acceleration Vector': [x * -1 for x in imu_data.col_values(55)],
                     'Thigh W Quaternion': imu_data.col_values(57),
                     'Thigh X Quaternion': imu_data.col_values(58),
                     'Thigh Y Quaternion': imu_data.col_values(59),
                     'Thigh Z Quaternion': imu_data.col_values(60),
                     'Weight': [weight] * num_data_points,
                     'Height': [height] * num_data_points,
                     'Floor to Shank IMU': [floor_to_shank_imu] * num_data_points,
                     'Floor to Knee': [floor_to_knee] * num_data_points,
                     'Floor to Hip': [floor_to_hip] * num_data_points,
                     'Knee to Hip IMU': [knee_to_hip_imu] * num_data_points
                    }
    for k in compiled_data:
        print(str(k) + " - " + str(len(compiled_data[k])))
    df = pd.DataFrame(compiled_data)
    return df, num_data_points

In [3]:
for i in os.listdir(os.getcwd() + "/ml_data"):
    if i[0] is not '~':
        df, num_data_points = process_file(os.getcwd() + "\\ml_data\\" + i)

C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Kyu_Alex_Walking.xlsx
Time - 1640
Time Between Readings - 1640
Shank X Gravity Vector - 1640
Shank Y Gravity Vector - 1640
Shank Z Gravity Vector - 1640
Shank X Gyroscope Vector - 1640
Shank Y Gyroscope Vector - 1640
Shank Z Gyroscope Vector - 1640
Shank X Magnetometer Vector - 1640
Shank Y Magnetometer Vector - 1640
Shank Z Magnetometer Vector - 1640
Shank X Euler Vector - 1640
Shank Y Euler Vector - 1640
Shank Z Euler Vector - 1640
Shank X Acceleration Vector - 1640
Shank Y Acceleration Vector - 1640
Shank Z Acceleration Vector - 1640
Shank X Linear Acceleration Vector - 1640
Shank Y Linear Acceleration Vector - 1640
Shank Z Linear Acceleration Vector - 1640
Shank W Quaternion - 1640
Shank X Quaternion - 1640
Shank Y Quaternion - 1640
Shank Z Quaternion - 1640
Thigh X Gravity Vector - 1640
Thigh Y Gravity Vector - 1640
Thigh Z Gravity Vector - 1640
Thigh X Gyroscope Vector - 1640
Thigh Y Gyroscope Vector - 164

In [4]:
import numpy as np
import math

In [5]:
def mult_quat(q1, q2):
    w = q1[0]*q2[0] - np.dot(q1[1], q2[1])
    v = np.add(np.cross(q1[1], q2[1]), q1[0]*q2[1])
    v = np.add(v, q2[0]*q1[1])
    return (w, v)
               
def conj_quat(quat):
    return (quat[0], -1*quat[1])

def inv_quat(quat):
    conjugate = conj_quat(quat)
    norm = math.sqrt(quat[0]**2 + quat[1][0]**2 + quat[1][1]**2 + quat[1][2]**2)
    return (conjugate[0] / norm, conjugate[1] / norm)

def rotate_quat(vect, rot_vect):
    rot_vect_conj = inv_quat(rot_vect)
    rotated_vect = mult_quat(rot_vect, vect)
    rotated_vect = mult_quat(rotated_vect, rot_vect_conj)
    return rotated_vect

def knee_angle_deg(thigh_quat, shank_quat):
    conjugate_thigh = conj_quat(thigh_quat)
    relative_quat = mult_quat(conjugate_thigh, shank_quat)
    knee_angle_w = math.acos(relative_quat[0])*360/math.pi
    return knee_angle_w  # is it possible to extract sin component from unit vector and then average angle from that?
    

In [6]:
angles = []
for i in range(num_data_points):
    shank_quat = (df['Shank W Quaternion'][i], np.array([df['Shank X Quaternion'][i], df['Shank Y Quaternion'][i], df['Shank Z Quaternion'][i]]))
#     print(shank_quat)
    thigh_quat = (df['Thigh W Quaternion'][i], np.array([df['Thigh X Quaternion'][i], df['Thigh Y Quaternion'][i], df['Thigh Z Quaternion'][i]]))
#     print(thigh_quat)
    rel_quat = mult_quat(inv_quat(thigh_quat), shank_quat)
#     print(rel_quat)
    angle = 90-360/math.pi*math.acos(rel_quat[0])
    print(str(i) + " - " + str(angle))
    angles.append(angle)
# print(angles)
df['PredictedKneeAngle'] = angles

0 - 41.292032250708914
1 - 14.057110609473611
2 - 14.924506708977205
3 - 15.232327235992997
4 - 16.202975891429148
5 - 16.57631200066679
6 - 20.10238130225376
7 - 22.822470588524865
8 - 24.964798870099088
9 - 31.054631746354772
10 - 39.980149020570806
11 - 46.45864748719911
12 - 55.04407919328644
13 - 54.48538017988711
14 - 56.09459862593308
15 - 55.96083503022336
16 - 52.87194238530758
17 - 47.87505376459084
18 - 42.11127231793923
19 - 32.199561633370294
20 - 23.436482929669168
21 - 16.765579893640847
22 - 12.640402610652586
23 - 13.449454039399768
24 - 22.401608517291365
25 - 24.013919788631895
26 - 25.664452869566446
27 - 29.339882901018292
28 - 28.15379952754946
29 - 25.531780498829107
30 - 23.136907056829983
31 - 22.59924207482659
32 - 19.873862404160747
33 - 17.02978004268803
34 - 14.631534216419965
35 - 13.209989308857402
36 - 12.544126189125677
37 - 12.348239695390163
38 - 15.088003840129375
39 - 19.20527682972117
40 - 23.61622681698259
41 - 29.600539242948933
42 - 36.747667039

522 - 38.3909032984775
523 - 29.382130416109888
524 - 22.138932539343713
525 - 20.52727064872154
526 - 25.042027889790234
527 - 27.381895710343784
528 - 33.264502305933306
529 - 35.66179795198317
530 - 31.985455920581863
531 - 27.176831556126935
532 - 28.43925043963587
533 - 23.464018222730246
534 - 21.460625895019334
535 - 21.03765357301033
536 - 18.904759828810484
537 - 16.841811451367974
538 - 17.573133378262412
539 - 17.547738065027048
540 - 21.531958010866816
541 - 22.591205002495315
542 - 27.573462720011925
543 - 35.87780743108246
544 - 45.83541663604503
545 - 53.297644298503684
546 - 59.44064809035048
547 - 64.73013432760305
548 - 68.20930694016799
549 - 67.72372242446701
550 - 63.4078642197909
551 - 55.74336044312361
552 - 51.760391571880795
553 - 43.604781852943404
554 - 32.94888766190489
555 - 25.411412768551514
556 - 21.57037635871079
557 - 21.954675318660236
558 - 24.354426206479587
559 - 30.092603815671296
560 - 34.77763151146727
561 - 32.61523655565425
562 - 28.4136930587

989 - 24.40465706399742
990 - 19.788486575071502
991 - 17.92166152910704
992 - 24.83491761170677
993 - 26.9455365635707
994 - 30.228346393202116
995 - 30.064612983275026
996 - 25.333802866196777
997 - 20.577789776517804
998 - 18.795528922568877
999 - 15.851988374765597
1000 - 14.770766644398975
1001 - 13.33697326321132
1002 - 13.980207058602247
1003 - 12.090745177710005
1004 - 13.446743865114215
1005 - 15.046499994542259
1006 - 15.253376389798277
1007 - 18.472909815900238
1008 - 24.565685773098053
1009 - 31.961694972694602
1010 - 41.18168648961037
1011 - 48.53221680687908
1012 - 52.855562027376166
1013 - 54.82277900371747
1014 - 56.85965268796085
1015 - 57.64796639144183
1016 - 55.14888107559994
1017 - 48.224995815013656
1018 - 40.693258165101895
1019 - 32.68177377821782
1020 - 18.417665728588105
1021 - 13.653261902069517
1022 - 13.74411496089526
1023 - 18.34070276761709
1024 - 16.321883384624428
1025 - 23.708552328512923
1026 - 28.704241270378922
1027 - 22.344086702272463
1028 - 20.80

1498 - 24.77774010506512
1499 - 23.04271494684083
1500 - 22.16374592513924
1501 - 22.129824860616452
1502 - 21.58170194487178
1503 - 21.972810720527505
1504 - 26.79735357649451
1505 - 28.427894334527288
1506 - 35.00236673760875
1507 - 43.4594180588473
1508 - 49.03873688323071
1509 - 50.06116995257741
1510 - 51.24411739316195
1511 - 51.80030291463494
1512 - 54.64900469134436
1513 - 53.30988562061712
1514 - 50.30589929565603
1515 - 46.25153111932572
1516 - 41.29740313837542
1517 - 30.876493502561367
1518 - 25.63416398735616
1519 - 20.625287390066845
1520 - 19.322514796986525
1521 - 19.71088268839233
1522 - 21.744105283454843
1523 - 24.92060657672735
1524 - 26.659534380600284
1525 - 21.16497024511277
1526 - 19.834649086916187
1527 - 18.467021152373675
1528 - 16.0121760241465
1529 - 13.34215025363443
1530 - 11.883525299677999
1531 - 10.350754219163534
1532 - 10.207338549393128
1533 - 9.513457276678238
1534 - 10.48700893100147
1535 - 11.98366623499659
1536 - 14.25573445976346
1537 - 20.3021

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


y = df.PredictedKneeAngle
features = ['Thigh X Gravity Vector', 'Thigh Y Gravity Vector', 'Thigh Z Gravity Vector', 
            'Thigh X Gyroscope Vector', 'Thigh Y Gyroscope Vector', 'Thigh Z Gyroscope Vector',
           'Thigh X Linear Acceleration Vector', 'Thigh Y Linear Acceleration Vector', 'Thigh Z Linear Acceleration Vector']
X = df[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

rf_angle_model = RandomForestRegressor(random_state=1, n_estimators=100)
rf_angle_model.fit(train_X, train_y)

# Calculate the mean absolute error of your Random Forest model on the validation data
rf_val_mae = mean_absolute_error(rf_angle_model.predict(val_X), val_y)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

Validation MAE for Random Forest Model: 4.35260430827787


In [9]:
# from micromlgen import port
# print(port(rf_angle_model))
import emlearn

filename = 'walking_model.h'
cmodel = emlearn.convert(rf_angle_model)
code = cmodel.save(file=filename)

print('Wrote C code to', filename)

ValueError: Unknown model type: 'RandomForestRegressor'

In [56]:
unit_quat = (1, np.array([0, 0, 0]))
quat0 = (0.66, np.array([0.57, -0.35, -0.34]))
quat_roll_90 = (np.cos(math.pi/4), np.array([np.sin(math.pi/4), 0, 0]))
quat_pitch_90 = (np.cos(math.pi/4), np.array([0, np.sin(math.pi/4), 0]))
quat_yaw_90 = (np.cos(math.pi/4), np.array([0, 0, np.sin(math.pi/4)]))
print(quat_roll_90)

(0.7071067811865476, array([0.70710678, 0.        , 0.        ]))


In [100]:
# rot_90 = rotate_quat(quat0, quat_roll_90)
# rot_180 = rotate_quat(rot_90, quat_roll_90)
# print(quat0)
# print(rot_90)
# print(rot_180)

# rot_90 = rotate_quat(rotate_quat(unit_quat, quat_roll_90), conj_quat(quat0))
# print(rot_90)
# print(rotate_quat(unit_quat, quat0))
# print(rotate_quat(unit_quat, quat_roll_90))
# print(mult_quat(conj_quat()))

offset_quat_shank0 = (0.65, np.array([-0.65, 0.25, -0.29]))
offset_quat_thigh0 = (0.66, np.array([0.23, -0.67, 0.23]))
offset_quat_shank1 = (0.46, np.array([-0.46, -0.58, 0.49]))
offset_quat_thigh1 = (-0.07, np.array([0.69, 0.06, 0.72]))
offset_quat_shank2 = (0.67, np.array([-0.64, 0.28, -0.24]))
offset_quat_thigh2 = (0.33, np.array([0.64, -0.37, 0.59]))
offset_quat_shank3 = (0.66, np.array([-0.71, 0.17, -0.18]))
offset_quat_thigh3 = (0.59, np.array([0.71, 0.09, 0.38]))

offset_quat_thigh_rel_to_shank0 = mult_quat(offset_quat_shank0, inv_quat(offset_quat_thigh0))
print(offset_quat_thigh_rel_to_shank0)
offset_quat_thigh_rel_to_shank1 = mult_quat(offset_quat_shank1, inv_quat(offset_quat_thigh1))
print(offset_quat_thigh_rel_to_shank1)
offset_quat_thigh_rel_to_shank2 = mult_quat(offset_quat_shank2, inv_quat(offset_quat_thigh2))
print(offset_quat_thigh_rel_to_shank2)
offset_quat_thigh_rel_to_shank3 = mult_quat(offset_quat_shank3, inv_quat(offset_quat_thigh3))
print(offset_quat_thigh_rel_to_shank3)

print()

offset_quat_shank1_rel_to_shank0 = mult_quat(offset_quat_shank0, inv_quat(offset_quat_shank1))
print(offset_quat_shank1_rel_to_shank0)
offset_quat_shank2_rel_to_shank0 = mult_quat(offset_quat_shank0, inv_quat(offset_quat_shank2))
print(offset_quat_shank2_rel_to_shank0)
offset_quat_shank3_rel_to_shank0 = mult_quat(offset_quat_shank0, inv_quat(offset_quat_shank3))
print(offset_quat_shank3_rel_to_shank0)

print()

offset_quat_thigh1_rel_to_thigh0 = mult_quat(offset_quat_thigh0, inv_quat(offset_quat_thigh1))
print(offset_quat_thigh1_rel_to_thigh0)
offset_quat_thigh2_rel_to_thigh0 = mult_quat(offset_quat_thigh0, inv_quat(offset_quat_thigh2))
print(offset_quat_thigh2_rel_to_thigh0)
offset_quat_thigh3_rel_to_thigh0 = mult_quat(offset_quat_thigh0, inv_quat(offset_quat_thigh3))
print(offset_quat_thigh3_rel_to_thigh0)

print()

offset90 = mult_quat((0.70, np.array([-0.71, -0.02, 0.02])), inv_quat((0.65, np.array([-0.69, -0.23, 0.23]))))
print(offset90)
# print(mult_quat(offset_quat_thigh_rel_to_shank, offset_quat_thigh))
# print(mult_quat(conj_quat(quat_roll_90), unit_quat))
# print(rotate_quat(conj_quat(mult_quat(conj_quat(quat_roll_90), unit_quat)), quat_roll_90))

(0.04552131638452683, array([-0.44385796,  0.52022926, -0.72241224]))
(-0.03155270638407299, array([ 0.16155784, -0.65531776, -0.73699533]))
(-0.43294301151621273, array([-0.71514958,  0.11609701, -0.53117126]))
(-0.16740705459132654, array([-0.96603248, -0.10086325, -0.17199628]))

(0.310946645495499, array([ 0.04570686,  0.04010602, -0.94404162]))
(0.9938368195693014, array([-0.04081239, -0.04422178, -0.01634501]))
(0.9847077691423941, array([ 0.02818591, -0.03438281, -0.14132935]))

(0.23754395091047367, array([ 0.02466303,  0.01417875, -0.96595216]))
(0.7472933788818003, array([-0.03623664,  0.01157975, -0.65605291]))
(0.5784422541838564, array([-0.05746512, -0.52935747, -0.61006802]))

(0.9520078814652552, array([ 0.02145286, -0.00149671, -0.29684765]))
